# Classificação

Instalando requisitos

In [80]:
%pip install -r ./app/requirements.txt -q
%pip install pandas -q
%pip install scikit-learn -q
%pip install tensorflow -q
%pip install numpy -q

You should consider upgrading via the '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You 

Importando requisitos

In [81]:
from app.modelo.imagem_rgb import ImagemRGB
import sklearn.model_selection
import pandas
import tensorflow
import os
import numpy

Definindo variáveis

In [82]:
image_directory = os.path.join(os.getcwd(), 'dataset_converted')
num_classes = 6

Abrindo base de dados

In [83]:
data_frame = pandas.read_csv("classifications.csv")

In [84]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y
0,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,1,SCC,792,462
1,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,2,SCC,601,678
2,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,3,SCC,363,467
3,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,4,SCC,599,437
4,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,5,Negative for intraepithelial lesion,1186,450
...,...,...,...,...,...,...,...
11529,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11530,LSIL,618,407
11530,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11531,LSIL,607,374
11531,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11532,LSIL,707,251
11532,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11533,LSIL,579,246


In [85]:
data_frame.shape

(11534, 7)

Separando o x

In [86]:
x = []
for index, row in data_frame.iterrows():
    x.append(os.path.join(image_directory, row['bethesda_system'], "{}.png".format(row['cell_id'])))
x = pandas.array(x)

In [87]:
x.shape

(11534,)

Separando o y

In [104]:
y = data_frame.iloc[:, 4].values
label_encoder = sklearn.preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [105]:
type(y)

numpy.ndarray

In [106]:
y

array([5, 5, 5, ..., 3, 3, 4])

Separando o conjunto de treino e teste

In [107]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

In [108]:
x_train[0]

'/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_05_27/dataset_converted/HSIL/8368.png'

In [109]:
y_train[1]

1

# Aplicando o ResNet50

Construindo o modelo

In [110]:
input_tensor = tensorflow.keras.layers.Input(shape=(100, 100, 3))
modelo_base = tensorflow.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
mbo = modelo_base.output
mbo = tensorflow.keras.layers.GlobalAveragePooling2D()(mbo)
mbo = tensorflow.keras.layers.Dense(1024, activation='relu')(mbo)
predicoes = tensorflow.keras.layers.Dense(num_classes, activation='softmax')(mbo)

modelo = tensorflow.keras.models.Model(inputs=modelo_base.input, outputs=predicoes)

for layer in modelo_base.layers:
    layer.trainable = False

Compilando

In [125]:
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'recall', 'precision'])

Treinando o modelo

In [126]:
def preprocessar_imagem(imagem):
    if isinstance(imagem, numpy.ndarray):
        imagem = imagem/255.0
    else:
        raise ValueError("Imagem deve ser um numpy array")
    
    imagem = tensorflow.convert_to_tensor(imagem, dtype=tensorflow.float32)
    imagem = tensorflow.expand_dims(imagem, axis=0)
    return imagem

In [133]:
for index, image_path in enumerate(x_train):
    if (os.path.exists(image_path)):
        imagem_rgb = ImagemRGB.from_file(image_path)
        imagem_preprocessada = preprocessar_imagem(imagem_rgb.matriz)
        label = tensorflow.keras.utils.to_categorical(y_train[index], num_classes)
        label = tensorflow.convert_to_tensor(label, dtype=tensorflow.int8)
        label = tensorflow.expand_dims(label, axis=0)
        modelo.fit(x=numpy.array(imagem_preprocessada), y=label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.0000e+00 - loss: 3.2265 - precision: 0.0000e+00 - recall: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 1.0000 - loss: 0.5294 - precision: 1.0000 - recall: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0000e+00 - loss: 3.3644 - precision: 0.0000e+00 - recall: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 1.0000 - loss: 0.6589 - precision: 1.0000 - recall: 1.0000


KeyboardInterrupt: 